In [3]:
import csv
import sys
import numpy
import pandas as pd

### PART 1:
Read in the file with csv.reader() and store it in an object called 'file_nested_list'.
Hint: This is a TSV file, and csv.reader() needs to be told how to handle it.
      https://docs.python.org/2/library/csv.html

In [72]:
with open("/home/anna/DAT-DC-10/data/chipotle.tsv", 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    l = [row for row in reader]

In [55]:
# PANDAS:
# Do I need to use with for this?  Or does pandas.read_table take care of f.close()?
chip = pd.read_table("/home/anna/DAT-DC-10/data/chipotle.tsv", sep = '\t')

### PART 2: 
Separate 'file_nested_list' into the 'header' and the 'data'.

In [144]:
file_nested_list = l
header = file_nested_list[0]
data = file_nested_list[1:]
header

['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']

In [74]:
print len(header), len(data), len(file_nested_list)

5 4622 4623


In [10]:
# PANDAS:
chip[0:3]

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39


### PART 3: 

Calculate the average price of an order.
Hint: Examine the data to see if the 'quantity' column is relevant to this calculation.
Hint: Think carefully about the simplest way to do this!

* count the number of unique order_id's.  note: you could assume this is 1834 since that's the maximum order_id, but it's best to check
* create a list of prices.  note: ignore the 'quantity' column because the 'item_price' takes quantity into account
* calculate the average price of an order and round to 2 digits

In [157]:
# Answer #1:
# I initially interpreted the question to be the average price per each order in a list (i.e. 1834)
# Using list comprehension after discussion in class:
l = []
for i in range(1, int(data[-1][0])+1):
    a = [float(price[1:-1]) for id,_,name,_,price in data if int(id) == i]
    l.append([i, round(numpy.sum(a),2)])
# Printing out the first five and last five entries
print "first five order sum = ", l[0:6], "\nlast five order sum = ", l[-5:]
# Printing out average of the order sums
print "Average of all order sums= $%.2f" %(numpy.sum([avg_per_ord for _,avg_per_ord in l])/float(len(l)))

first five order sum =  [[1, 11.56], [2, 16.98], [3, 12.67], [4, 21.0], [5, 13.7], [6, 17.5]] 
last five order sum =  [[1830, 23.0], [1831, 12.9], [1832, 13.2], [1833, 23.5], [1834, 28.75]]
Average of all order sums= $18.81


In [99]:
# Answer #2
# In class we did this: easiest way is to add all prices and divide across number of orders
total_price = numpy.sum([float(price[1:]) for _,_,_,_,price in data])
print "totaled price= %.2f\norders= %d\navg= %.2f" %(total_price,int(data[-1][0]), total_price/float(data[-1][0]))

totaled price= 34500.16
orders= 1834
avg= 18.81


In [77]:
# Answer #3:
# This was my first attempt without using much list comprehensions with nested loops
# I also mis-interpreted the question to be an average within an order_id and then
# averaging across the averages.
row = avg = 0
ord_sum = []
for id in range(1, int(data[len(data)-1][0]) + 1): # 1834 + 1
    qty = tot = 0.0
    while (int(data[row][0]) == id):
        tot = tot + float(data[row][4].strip(" ").strip("$"))
        qty = qty + int(data[row][1])
        avg = tot / qty  # seems like you have to use qty to get avg.
        row = row + 1
        if row >= len(data):
            break;
        
    tup = (id, float(format(tot, ".2f")), float(format(avg, ".2f")))
    ord_sum.append(tup)

print ord_sum[1830:1835]
print "average of sums= %.2f" %(numpy.sum([o[1] for o in ord_sum]) / int(data[-1][0]))
print "average of avgs= %.2f" %(numpy.sum([o[2] for o in ord_sum]) / int(data[-1][0]))


[(1831, 12.9, 4.3), (1832, 13.2, 6.6), (1833, 23.5, 11.75), (1834, 28.75, 9.58)]
average of sums= 18.81
average of avgs= 7.21


In [98]:
# PANDAS:
chip_flt = chip["item_price"].str.replace('$','').apply(float)
round(float(sum(chip_flt) / chip[len(chip)-1:len(chip)].order_id.apply(int)), 2)
# Why can't I use -1 as an index in pandas data frame??
# In this case here, I don't think this was any easier than using list comprehension above.

18.81

### PART 4: 

Create a list (or set) of all unique sodas and soft drinks that they sell.
Note: Just look for 'Canned Soda' and 'Canned Soft Drink', and ignore other drinks like 'Izze'.

In [204]:
unique_sodas = set([i[3][1:-1] for i in data if "Canned" in i[2]]) 
unique_sodas

{'Coca Cola',
 'Coke',
 'Diet Coke',
 'Diet Dr. Pepper',
 'Dr. Pepper',
 'Lemonade',
 'Mountain Dew',
 'Nestea',
 'Sprite'}

In [134]:
# PANDAS:
bool_chip = chip.item_name.str.contains("Canned")
type(bool_chip) # This needs to be a Series, not a list for the statement below to work
set(chip[bool_chip].choice_description.str.strip("[]"))

{'Coca Cola',
 'Coke',
 'Diet Coke',
 'Diet Dr. Pepper',
 'Dr. Pepper',
 'Lemonade',
 'Mountain Dew',
 'Nestea',
 'Sprite'}

### PART 5: 

Calculate the average number of toppings per burrito.
Note: Let's ignore the 'quantity' column to simplify this task.
Hint: Think carefully about the easiest way to count the number of toppings!
* keep a running total of burritos and toppings, so initalize some counter variables
* calculate number of toppings by counting the commas and adding 1
* note: x += 1 is equivalent to x = x + 1
* loop over the data and update the counters in this way
* calculate the average topping count and round to 2 digits

In [145]:
# Answer #1: Using list comprehension 
n_tops = 0
l = [(desc.count(',')+1) for _,_,name,desc,_ in data if "Burrito" in name]
n_tops = numpy.sum(l)
n_burrs = len(l)

print "toppings= %d burritos= %d avg= %.3f" %(n_tops, n_burrs, float(n_tops)/n_burrs)

toppings= 6323 burritos= 1172 avg= 5.395


In [193]:
# PANDAS:
chip["commas"] = chip.choice_description.str.count(',') + 1
burrito = chip[chip.item_name.str.contains("Burrito")]
# Note that the statement below was disliked by pandas, because it is a slice of the orig.
# burrito["commas"] = burrito.choice_description.str.count(',') + 1
# How to format the burrito.groupby("item_name").commas.mean() Series? I used numpy.round()
print "Average # toppings per Burrito type=", numpy.round(burrito.groupby("item_name").commas.mean(),3)
print "\nAverage of averages = %.3f" %(burrito.groupby("item_name").commas.mean().mean())
print "\nAverage across all  = %.3f" %(burrito.commas.mean())


Average # toppings per Burrito type= item_name
Barbacoa Burrito    5.143
Burrito             5.833
Carnitas Burrito    5.373
Chicken Burrito     5.329
Steak Burrito       5.408
Veggie Burrito      5.958
Name: commas, dtype: float64

Average of averages = 5.507

Average across all  = 5.395


In [195]:
# Answer #2:  This was my attempt before class which attempts to average # of toppings per type of burrito
# A convoluted way of getting avg per type of Burrito and then calculating overall average using dictionary
# and nested loops and no list comprehension
import re
from __future__ import division
d_ords = {}
d_tops = {}
d_avg = {}
unique_b = set([name for _,_,name,_,_ in data if "Burrito" in name])
for item in data:
    if item[2] in unique_b:
        if item[2] in d_ords:
            d_ords[item[2]] = d_ords[item[2]] + 1
            d_tops[item[2]] = d_tops[item[2]] + len([m.start() for m in re.finditer(',', item[3])]) + 1
        else:
            d_ords[item[2]] = 1
            d_tops[item[2]] = len([m.start() for m in re.finditer(',', item[3])]) + 1

for i in d_ords:
    d_avg[i] = format(d_tops[i] / d_ords[i], ".2f")

In [196]:
d_ords

{'Barbacoa Burrito': 91,
 'Burrito': 6,
 'Carnitas Burrito': 59,
 'Chicken Burrito': 553,
 'Steak Burrito': 368,
 'Veggie Burrito': 95}

In [197]:
tot_ord = 0
for i in d_ords:
    tot_ord += d_ords[i]

print tot_ord

1172


In [198]:
d_tops

{'Barbacoa Burrito': 468,
 'Burrito': 35,
 'Carnitas Burrito': 317,
 'Chicken Burrito': 2947,
 'Steak Burrito': 1990,
 'Veggie Burrito': 566}

In [199]:
tot_top = 0
for i in d_tops:
    tot_top += d_tops[i]
print "total # of toppings:", tot_top
print "average # of toppings: %.2f" %(float(tot_top) / float(tot_ord))

total # of toppings: 6323
average # of toppings: 5.40


In [204]:
print d_avg
print "Average of averages: %.3f" %(sum([float(d_avg[a]) for a in d_avg]) / len(d_avg))

{'Burrito': '5.83', 'Chicken Burrito': '5.33', 'Barbacoa Burrito': '5.14', 'Veggie Burrito': '5.96', 'Steak Burrito': '5.41', 'Carnitas Burrito': '5.37'}
Average of averages: 5.507


### PART 6: 
Create a dictionary in which the keys represent chip orders and the values represent the total number of orders.
Expected output: {'Chips and Roasted Chili-Corn Salsa': 18, ... }
Note: Please take the 'quantity' column into account!
Optional: Learn how to use 'defaultdict' to simplify your code.

* start with an empty dictionary
* loop over the data rows and do the following - 
* if chip order is not in dictionary, then add a new key/value pair
* if chip order is already in dictionary, then update the value for that key
* let's learn about a new way to do this
* defaultdict saves you the trouble of checking whether a key already exists

In [201]:
# Answer 1: After class discussion
# Is there a way to do this with list comprehension when working with dictionary?
from collections import defaultdict
d = defaultdict(int)
for _,qty,name,_,_ in data 
    if 'Chip' in name:
        d[name] += int(qty)

d

defaultdict(int,
            {'Chips': 230,
             'Chips and Fresh Tomato Salsa': 130,
             'Chips and Guacamole': 506,
             'Chips and Mild Fresh Tomato Salsa': 1,
             'Chips and Roasted Chili Corn Salsa': 23,
             'Chips and Roasted Chili-Corn Salsa': 18,
             'Chips and Tomatillo Green Chili Salsa': 45,
             'Chips and Tomatillo Red Chili Salsa': 50,
             'Chips and Tomatillo-Green Chili Salsa': 33,
             'Chips and Tomatillo-Red Chili Salsa': 25,
             'Side of Chips': 110})

In [229]:
# PANDAS:
chip = pd.read_table("../data/chipotle.tsv", sep='\t')
boolean_series = chip.item_name.str.contains("Chips")
chip[chip.item_name.str.contains("Chips")].groupby("item_name").quantity.sum()

item_name
Chips                                    230
Chips and Fresh Tomato Salsa             130
Chips and Guacamole                      506
Chips and Mild Fresh Tomato Salsa          1
Chips and Roasted Chili Corn Salsa        23
Chips and Roasted Chili-Corn Salsa        18
Chips and Tomatillo Green Chili Salsa     45
Chips and Tomatillo Red Chili Salsa       50
Chips and Tomatillo-Green Chili Salsa     33
Chips and Tomatillo-Red Chili Salsa       25
Side of Chips                            110
Name: quantity, dtype: int64

In [109]:
# Answer 2:  Before class without list comprehension
from collections import defaultdict
d = defaultdict(int)
for item in data:
    found = len([m.start() for m in re.finditer('Chip', item[2])])
    if found > 0:
        d[item[2]] = d[item[2]] + int(item[1])
    

In [110]:
d

defaultdict(int,
            {'Chips': 230,
             'Chips and Fresh Tomato Salsa': 130,
             'Chips and Guacamole': 506,
             'Chips and Mild Fresh Tomato Salsa': 1,
             'Chips and Roasted Chili Corn Salsa': 23,
             'Chips and Roasted Chili-Corn Salsa': 18,
             'Chips and Tomatillo Green Chili Salsa': 45,
             'Chips and Tomatillo Red Chili Salsa': 50,
             'Chips and Tomatillo-Green Chili Salsa': 33,
             'Chips and Tomatillo-Red Chili Salsa': 25,
             'Side of Chips': 110})